<a href="https://colab.research.google.com/github/Vik7am10/Empty/blob/main/kagglecompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2, l1_l2
from tensorflow.keras.layers import Add
from tensorflow.keras.callbacks import Callback
import numpy as np

In [2]:
import zipfile
import os
import shutil

# Path to the ZIP file
zip_file_path = '/content/cs-480-2024-spring.zip'

# Directory to unzip to
unzip_dir = '/content/data'

# Final target directory
target_dir = '/content/target_folder'

# Step 1: Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Step 2: Move the unzipped contents to the target directory
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

for filename in os.listdir(unzip_dir):
    shutil.move(os.path.join(unzip_dir, filename), target_dir)

# Step 3: Clean up the temporary directory
shutil.rmtree(unzip_dir)



In [22]:
# Load CSV and TSV files
train_df = pd.read_csv('/content/target_folder/data/train.csv')
target_columns = [col for col in train_df.columns if 'X' in col]
ancillary_columns = [col for col in train_df.columns if col not in ['id'] + target_columns]

# Define the batch size
batch_size = 43363
num_batches = len(train_df) // batch_size + (1 if len(train_df) % batch_size != 0 else 0)
anc_size = len(ancillary_columns)
df_size = len(train_df)

print(f"Number of batches: {num_batches}")
print(f"Ancillary: {anc_size}")
print(f"DF: {df_size}")

Number of batches: 1
Ancillary: 151
DF: 43363


In [23]:
# Function to load and preprocess images
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [64, 64])  # Resize to 64x64 for ResNet50
    image = tf.image.random_flip_left_right(image)  # Random horizontal flip
    image = tf.image.random_flip_up_down(image)  # Random vertical flip
    image = tf.image.random_brightness(image, max_delta=0.1)  # Random brightness
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Random contrast
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)  # Random saturation
    image = tf.image.random_hue(image, max_delta=0.1)  # Random hue
    image = image / 255.0  # Normalize to [0, 1]
    return image.numpy()  # Convert to numpy array

# processing the first and only batch
batch_idx = 0
start_idx = batch_idx * batch_size
end_idx = min((batch_idx + 1) * batch_size, len(train_df))

batch_image_ids = train_df['id'].iloc[start_idx:end_idx]

# Load images for the current batch
image_paths = [os.path.join('/content/target_folder/data/train_images', f'{img_id}.jpeg') for img_id in batch_image_ids]
train_images = np.array([preprocess_image(path) for path in image_paths])

# Extract corresponding ancillary data and targets
X_ancillary_batch = train_df[ancillary_columns].iloc[start_idx:end_idx].values
y_train_batch = train_df[target_columns].iloc[start_idx:end_idx].values

# Normalize ancillary data
scaler_ancillary = StandardScaler()
X_ancillary_batch = scaler_ancillary.fit_transform(X_ancillary_batch)

# Normalize target data
scaler_target = MinMaxScaler()
y_train_batch = scaler_target.fit_transform(y_train_batch)

In [90]:
# Define the model architecture with minimal changes
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
base_model.trainable = True

inputs = Input(shape=(64, 64, 3))
x = base_model(inputs, training=True)
x = Flatten()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
image_features = Dense(64, activation='relu')(x)

ancillary_input = Input(shape=(X_ancillary_batch.shape[1],))
ancillary_features = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(ancillary_input)
ancillary_features = BatchNormalization()(ancillary_features)
ancillary_features = Dropout(0.35)(ancillary_features)

combined_features = Concatenate()([image_features, ancillary_features])
combined_features = Dense(64, activation='relu')(combined_features)
combined_features = Add()([combined_features, image_features])
combined_features = BatchNormalization()(combined_features)
combined_features = Dropout(0.35)(combined_features)
output = Dense(len(target_columns), activation='linear')(combined_features)

model = Model(inputs=[inputs, ancillary_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')  # Slightly increased learning rate

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, min_lr=1e-7, verbose=1)
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_57            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 2, 2, 2048)     │     23,587,712 │ input_layer_57[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_19 (Flatten)      │ (None, 8192)           │              0 │ resnet50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_97 (Dense)          │ (None, 256)            │      2,097,408 │ flatten_19[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_58            │ (None, 151)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_61    │ (None, 256)            │          1,024 │ dense_97[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_99 (Dense)          │ (None, 64)             │          9,728 │ input_layer_58[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_61 (Dropout)      │ (None, 256)            │              0 │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_62    │ (None, 64)             │            256 │ dense_99[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_98 (Dense)          │ (None, 64)             │         16,448 │ dropout_61[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_62 (Dropout)      │ (None, 64)             │              0 │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_16            │ (None, 128)            │              0 │ dense_98[0][0],        │
│ (Concatenate)             │                        │                │ dropout_62[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_100 (Dense)         │ (None, 64)             │          8,256 │ concatenate_16[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_5 (Add)               │ (None, 64)             │              0 │ dense_100[0][0],       │
│                           │                        │                │ dense_98[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_63    │ (None, 64)             │            256 │ add_5[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_63 (Dropout)      │ (None, 64)             │              0 │ batch_normalization_6… │
├──────────────────────

 Total params: 25,722,258 (98.12 MB)

 Trainable params: 25,668,370 (97.92 MB)

 Non-trainable params: 53,888 (210.50 KB)

In [91]:
# Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=20,  # Increased rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Training Data Generator
def train_generator(X_img, X_anc, y, batch_size=64):
    data_gen_flow = data_gen.flow(X_img, y, batch_size=batch_size, shuffle=True)
    while True:
        for X_batch, y_batch in data_gen_flow:
            start = 0
            end = len(X_batch)
            ancillary_batch = X_anc[start:end]
            yield (tf.convert_to_tensor(X_batch, dtype=tf.float32),
                   tf.convert_to_tensor(ancillary_batch, dtype=tf.float32)), \
                   tf.convert_to_tensor(y_batch, dtype=tf.float32)

# Prepare Dataset
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator(train_images, X_ancillary_batch, y_train_batch, batch_size=64),
    output_signature=(
        (tf.TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, X_ancillary_batch.shape[1]), dtype=tf.float32)),
        tf.TensorSpec(shape=(None, y_train_batch.shape[1]), dtype=tf.float32)
    )
)

# Train the Model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(train_images) // 64,
    epochs=15,
    callbacks=[reduce_lr, early_stop]
)

# Save model weights
model.save_weights(f"model_after_batch_{batch_idx}.weights.h5")

Epoch 1/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 113s 86ms/step - loss: 7.0722 - learning_rate: 1.0000e-04
Epoch 2/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 77s 85ms/step - loss: 3.4655 - learning_rate: 1.0000e-04
Epoch 3/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 58s 85ms/step - loss: 1.5564 - learning_rate: 1.0000e-04
Epoch 4/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 57s 85ms/step - loss: 0.7896 - learning_rate: 1.0000e-04
Epoch 5/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 58s 85ms/step - loss: 0.5041 - learning_rate: 1.0000e-04
Epoch 6/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 59s 87ms/step - loss: 0.3935 - learning_rate: 1.0000e-04
Epoch 7/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 58s 86ms/step - loss: 0.3126 - learning_rate: 1.0000e-04
Epoch 8/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 58s 86ms/step - loss: 0.2671 - learning_rate: 1.0000e-04
Epoch 9/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 58s 86ms/step - loss: 0.2223 - learning_rate: 1.0000e-04
Epoch 10/15
677/677 ━━━━━━━━━━━━━━━━━━━━ 59s 87ms/step - loss: 0.1908 - learning_rate: 1.0000e-04
Epoch 11/15
677/677 ━━━━━━━━

In [93]:
# Load the final trained model weights
model.load_weights(f"model_after_batch_0.weights.h5")

# Preprocess and load all test images
test_df = pd.read_csv('/content/target_folder/data/test.csv')
test_image_ids = test_df['id'].values
test_image_paths = [os.path.join('/content/target_folder/data/test_images', f'{img_id}.jpeg') for img_id in test_image_ids]
test_images = np.array([preprocess_image(path) for path in test_image_paths])

# Standardize ancillary data for test set
X_test_ancillary = test_df[ancillary_columns].values
X_test_ancillary = scaler_ancillary.transform(X_test_ancillary)

# Predict on the entire test set
predictions_log = model.predict([test_images, X_test_ancillary])

# Reverse the normalization on predictions
predictions = scaler_target.inverse_transform(predictions_log)

# Unnormalize the predictions if necessary
predictions_unnormalized = predictions

# Select the first and last 6 columns for the output
selected_columns = target_columns[:6] + target_columns[-6:]
predictions_selected = pd.DataFrame(predictions_unnormalized, columns=target_columns)[selected_columns]

# Add 'id' column for identification
predictions_selected['id'] = test_df['id']
predictions_selected = predictions_selected[['id'] + selected_columns]

# Save the selected predictions to a CSV file
predictions_selected.to_csv('predictions_selected.csv', index=False)

print("Predictions for all test images have been generated, unnormalized, and saved.")


200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Predictions for all test images have been generated, unnormalized, and saved.


In [94]:
df_filtered = predictions_selected.iloc[:, [0] + list(range(-6, 0))]
df_filtered.to_csv('submission.csv', index=False)